In [8]:
from core.libs import pd
from core.db import get_engine
from core.schema_helpers import rename_columns_using_schema
from CruisesProcessor.general_importer import prepare_df_for_sql, save_inventory_to_sql, ensure_table
from CruisesProcessor.dead_alive_calculator import calculate_dead_alive
from CruisesProcessor.tree_id import split_by_id_validity
from core.doyle_calculator import calculate_doyle

archivos = [
    r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Costa Rica\2021_ForestInventory\7-Database\Final\CostaRica_Database_All v1.1.xlsx",
    r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Mexico\2021_ForestInventory\6-Database\2021_Mexico_Database v1.0.xlsx"
    ,
    r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Guatemala\2021_ForestInventory\6-Database\Database\2021_Guatemala_Database v1.2.xlsx",
    r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\USA\2021_ForestInventory\9-Database\2021_US_Database v1.1.xlsx"
    ]
tablas_destino = ['inventory_cr_2021','inventory_mx_2021','inventory_gt_2021',  ]
sheet = 'TreeList_Data'

In [9]:
#Leer el archivo
df_original = pd.read_excel(archivo, sheet_name=sheet)
#print("📄 Columnas originales:", list(df_original.columns))

df = df_original.drop(columns=["DAP (cm)", "THt (m)", "MHt (m)"], errors="ignore")
#print("📄 Columnas actuales:  ", list(df.columns))

# Mostrar diferencia
eliminadas = set(df_original.columns) - set(df.columns)
#print("🗑️ Columnas eliminadas:", eliminadas)
df = rename_columns_using_schema(df)
#print(df.head(5))


In [10]:
# CALCULADORES que faltaban:
engine = get_engine()
df = calculate_dead_alive(df, engine)
df = calculate_doyle(df)

df_ok, df_bad = split_by_id_validity(df)

💻 Conectado a la base de datos helloworldtree
🌳 Doyle calculado


In [12]:
df_sql, _ = prepare_df_for_sql(df_ok)
df_sql = df_sql.where(pd.notna(df_sql), None)

#print(df_sql.head(5))

# ← El segundo valor (dtypes) lo ignoramos
tabla_destino = "inventory_cr_2022"

  contractcode        farmername cruisedate              id id_error  stand  \
0       CR0086  Agner Chinchilla       None  CR008601010001     None    1.0   
1       CR0086  Agner Chinchilla       None  CR008601010002     None    1.0   
2       CR0086  Agner Chinchilla       None  CR008601010003     None    1.0   
3       CR0086  Agner Chinchilla       None  CR008601010004     None    1.0   
4       CR0086  Agner Chinchilla       None  CR008601010005     None    1.0   

  plot plot_coordinate  tree_number defect_ht_ft    dbh_in   tht_ft  \
0  1.0            <NA>          1.0         None  1.102362  6.56168   
1  1.0            <NA>          2.0         None  1.062992  6.56168   
2  1.0            <NA>          3.0         None  0.905512  6.56168   
3  1.0            <NA>          4.0         None  1.377953  9.84252   
4  1.0            <NA>          5.0         None  0.393701  3.28084   

   merch_ht_ft short_note Species_id Defect_id Pests_id Coppiced_id  \
0          NaN       <NA>  

In [14]:
ensure_table(df_sql, engine, tabla_destino)
#print(df_sql.head(1))

In [15]:
save_inventory_to_sql(df_sql, engine, tabla_destino)




=== INICIO DE IMPORTACIÓN ===
💻 Conectado a la base de datos helloworldtree
✅ Bulk insert completado: 
 'inventory_cr_2022' (4273 filas)
